#  Exerciese 6-2

## Myokyung Han (20224230)

# Data Downloading

### Exchange Rate - krw_usd_xr.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A036Y001.0000001

### Current Account - Korea - current_account_korea.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A022Y013.000000

### Current Account - US - current_account_us.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y031.US

### GDP: Korea - gdp_korea.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y042.KOR

### GDP Growth korea - gdp_growth_korea.xlsx 
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A901Y001.AI1AA
    
### GDP: US - gdp_us.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y042.USA

### GDP Growth US - gdp_growth_us.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y041.USA

### Consumer Price Index: Korea - cpi_korea.xlsx >> non stationary
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y022.KR

### Consumer Price Index: US - cpi_us.xlsx >> non stationary
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y022.US

### 1-year Interest Rate: Korea - int_rate_korea.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A060Y001.010190000
m
### 1-year Interest Rate: US - int_rate_us.xlsx

https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=FRED%3ADGS1

### Foreign Reserves at Bank of Korea - foreign_reserves.xlsx >> non stationary
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A901Y001.AI1DC

## Data Loading

In [1]:
import pandas as pd

In [2]:
def load_deepsearch_data(filepath, column):
    df = pd.read_excel(filepath)
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.set_index('날짜')
    df.columns = [column]
    
    return df

In [3]:
filenames = [
    'krw_usd_xr',
    'current_account_korea', 'current_account_us',
    'gdp_korea', 'gdp_us',
    'gdp_growth_korea', 'gdp_growth_us',
    'cpi_korea', 'cpi_us',
    'int_rate_korea', 'int_rate_us',
    'foreign_reserves',
]

In [4]:
dfs = []
for fname in filenames:
    print('Loading {}...'.format(fname))
    # 짱신기해...
    # filepath = './data/{}.xlxs'.format(fname)와 동일
    filepath = f'../data/{fname}.xlsx'
    
    df = load_deepsearch_data(filepath, fname)
    
    if fname in ['gdp_growth_korea', 'gdp_growth_us', 'current_account_us']:
        df[fname] = df[fname].rolling(4).sum()
    elif fname in ['current_account_korea']:
        df[fname] = df[fname].rolling(12).sum()
        
    
    dfs.append(df)
    
print('Data loading is completed')

Loading krw_usd_xr...
Loading current_account_korea...
Loading current_account_us...
Loading gdp_korea...
Loading gdp_us...
Loading gdp_growth_korea...
Loading gdp_growth_us...
Loading cpi_korea...
Loading cpi_us...
Loading int_rate_korea...
Loading int_rate_us...
Loading foreign_reserves...
Data loading is completed


In [5]:
from pandas.tseries.offsets import YearEnd

In [6]:
def load_world_bank_data(filepath, column):
    df = pd.read_excel(filepath, sheet_name = 'Data', skiprows = 3)
    
    df = df.set_index('Country Code')
    df = df.loc['KOR'].to_frame()
    
    df = df.iloc[3:]
    df.dropna(inplace=True)
    df['KOR'] = df['KOR'].astype(float)
    df['date'] = pd.to_datetime(df.index)+YearEnd(0)
    df.set_index('date', inplace=True)
    df.columns = [column]
    return df
      
    

In [7]:
df_ppp_xr = load_world_bank_data('./data/world_ppp_xr.xls', 'krw_usd_ppp_xr')

FileNotFoundError: [Errno 2] No such file or directory: './data/world_ppp_xr.xls'

In [ ]:
dfs.append(df_ppp_xr)

In [ ]:
len(dfs)

In [ ]:
df_merged = pd.concat(dfs, axis = 1)

In [ ]:
df_merged = df_merged.resample('M').last()

In [ ]:
df_merged = df_merged.fillna(method='ffill')

In [ ]:
df_merged['2010-01-01':].head()

In [ ]:
df_merged['2010-01-01':].tail()

## Compute new variables

### Currency returens

In [ ]:
df_merged['ann_ret']  = df_merged['krw_usd_xr'] .pct_change(periods=12)*100

In [ ]:
df_merged['F12.ann_ret'] = df_merged['ann_ret'].shift(-12)

### Real XR

In [ ]:
df_merged['real_xr'] = df_merged['krw_usd_ppp_xr']  /df_merged['krw_usd_xr']  

### current account relative to GDP
non_stationary!!

In [ ]:
df_merged['ca_gdp_korea'] = df_merged['current_account_korea']/df_merged['gdp_korea']*100

In [ ]:
df_merged['ca_gdp_us'] = df_merged['current_account_us']/df_merged['gdp_us']*100

In [ ]:
df_merged.loc['2019-10-01':][['ca_gdp_korea', 'ca_gdp_us']].head()

### GDP Growth

In [ ]:
df_merged['gdp_growth_korea2'] = df_merged['gdp_korea'].pct_change(periods=12)*100

In [ ]:
df_merged['gdp_growth_us2'] = df_merged['gdp_us'].pct_change(periods=12)*100

In [ ]:
df_merged.loc[['2018-12-31', '2019-12-31']][['gdp_us', 'gdp_growth_us','gdp_growth_us2']]

In [ ]:
# why is it differnece!!!!
# https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=US
# Current US 달러이냐 constant us dollar이냐에 따라 달라진다. 
# Current US>> market
# constant us dollar >> inflation.... 2015년 기준으로 보았을때 2.5가 된다..

### Inflation

In [ ]:
df_merged['infl_korea'] = df_merged['cpi_korea'].pct_change(periods=12)*100

In [ ]:
df_merged['infl_us'] = df_merged['cpi_us'].pct_change(periods=12)*100

### Inflation Interest rate differential

In [ ]:
df_merged['infl_diff'] = df_merged['infl_korea']- df_merged['infl_us']

In [ ]:
df_merged['int_rate_diff'] = df_merged['int_rate_korea']- df_merged['int_rate_us']

### Foreign Reserves

In [ ]:
df_merged['reserves_to_gdp'] = df_merged['foreign_reserves']/df_merged['gdp_korea']/1000*100
# 1000 >>단위때문에 곱하고

In [ ]:
df_merged['reserves_growth'] = df_merged['foreign_reserves'].pct_change(periods=12)*100
# 다시 들어야겠따...
# trand how it is changing ......